In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy import stats
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import gc
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold,StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.externals import joblib
from sklearn.svm import SVC
import sklearn.metrics as metrics
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

path = './data/Debt issuing company 2018 report/'

# 去掉异常值

In [2]:
def drop_out(frame,col,model='Confidence interval',t_alpha=0.95,alpha=2,IQR_rate=1.5,quantile=0.95):
    '''modle: 'gauss','box','quantile' '''
    
    if model == 'Confidence interval':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        interval_ = stats.t.interval(t_alpha,frame[col].count()-1,u_,v_)
        cond_ = (frame[col]<interval_[1])&(frame[col]>interval_[0])
    
    elif model == 'gauss':
        u_ = frame[col].mean()
        v_ = frame[col].std()
        cond_ = np.abs((frame[col]-u_)/v_) < alpha
    
    elif model == 'box':
        q1 = frame[col].quantile(0.25)
        q3 = frame[col].quantile(0.75)
        IQR = (q3-q1)*IQR_rate
        q1 -= IQR ; q3 += IQR
        cond_ = (frame[col]<q3)&(frame[col]>q1)
    
    elif model == 'quantile':
        top_ = frame[col].quantile(quantile)
        bottom_ = frame[col].quantile(1-quantile)
        cond_ = (frame[col]<top_)&(frame[col]>bottom_)
    
    else:
        print('please try again')
        return frame
    
    index_ = np.where(frame[col]!=frame[col],True,
                                                  np.where(cond_,True,False))
    frame = frame.loc[index_,:]
    return frame

# 时间操作

In [3]:
def mms(frame,groupby,col,):
    max_ = dict(frame.groupby(by=groupby,)[col].max())
    min_ = dict(frame.groupby(by=groupby,)[col].min())
    max_ = frame[groupby].map(lambda x:max_[x])
    min_ = frame[groupby].map(lambda x:min_[x])
    result = (frame[col].values - min_)/(max_-min_)
    return [round(i,2) for i in result]

def moving(frame,groupby,time_col,col,wins=3,weight=[0.55,0.3,0.15]):
    '''wins是窗口数,weight是权重,这两个必须对应'''
    frame = frame.sort_values([groupby,time_col])
    for i in range(wins):
        if i == 0 :
            list_ = weight[i]*(frame[col].values)
        else :
            list_ += weight[i]*(frame.groupby(groupby)[col].rolling(i+1).mean().values)
    return list_

def bodonglv(frame,groupby,time_col,col,wins):
    frame = frame.sort_values([groupby,time_col])
    std_ = (frame.groupby(groupby)[col].rolling(wins).std().values)
    mean_ = (frame.groupby(groupby)[col].rolling(wins).mean().values)
    return [round(std_[i]/mean_[i],2) for i in range(len(std_))]

def tongbi(frame,groupby,time_col,col,diff):
    frame = frame.sort_values([groupby,time_col])
    diff_ = frame.groupby(groupby)[col].diff(diff).values
    ori_ = frame[col].values
    return [np.nan]*diff+[round(diff_[i]/abs(ori_[i-diff]),2) for i in range(diff,frame.shape[0])]

# 补充空白值

In [4]:
def diy_ss(frame,weight_dict,quantile=0.2):
    for i in frame.columns:
        if frame[i].min() >= 0:
            frame[i] = np.log1p(frame[i])
            weight_dict[i] = 'log1p'
        else :
            mean_ = frame[i][(frame[i]>=frame[i].quantile(quantile)) & (frame[i]<=frame[i].quantile(1-quantile))].mean()
            std_ = frame[i][(frame[i]>=frame[i].quantile(quantile)) & (frame[i]<=frame[i].quantile(1-quantile))].std()
            weight_dict[i] = [mean_,std_]
            frame[i] = (frame[i]-mean_)/std_
    return frame,weight_dict

In [5]:
def fillna_(frame,group_col,quantile=0.2,n_epoch=3):
    
    def return_index(aa,bb):
        j=0
        cc = []
        for i in range(len(aa)):
            if aa[i] == False :
                cc.append(aa[i])
            else:
                cc.append(bb[j])
                j += 1
        return cc
    
    frame_col = frame.columns
    
    for l,comp in enumerate(set(group_col)):

        index_y = list(group_col == comp)
        full_col = []
        loss_col = {}

        for col in frame.columns:
            if frame.loc[index_y,col].isnull().sum() == 0:
                full_col.append(col)
            else:
                loss_col[col] = frame.loc[index_y,col].isnull().sum()

        loss_col = sorted(loss_col.items(),key=lambda x:x[1])
        loss_col = [i[0] for i in loss_col]
        
        index_dict = {}
        if len(full_col) == 0:
            index_dict[loss_col[0]] = frame.loc[index_y,loss_col[0]].isnull()
            index_dict[loss_col[0]].fillna(index_dict[loss_col[0]].median(),inplace=True)
            full_col.append(loss_col[0])
            loss_col = loss_col[1:]
            
        for epoch in range(n_epoch):

            if epoch == 0:
                for _,col in enumerate(loss_col):
                    if np.random.rand()>0.75:
                        print(comp,f'{l}/{len(set(group_col))}',col,f'{_}/{len(loss_col)}')
                    index_l = list(frame.loc[index_y,col].isnull())
                    index_f = list(frame.loc[index_y,col].notnull())
                    index_l_ = return_index(index_y,index_l)
                    index_f_ = return_index(index_y,index_f)
                    index_dict[col] = (index_l_,index_f_)
                    rfr = RandomForestRegressor(n_estimators=10,n_jobs=-1,max_features=0.9)
                    rfr.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                    pre = rfr.predict(frame.loc[index_l_,full_col])
                    frame.loc[index_l_,col] = pre
                    full_col.append(col)
            
            else:
                for col in index_dict:
                    index_l_ = index_dict[col][0]
                    index_f_ = index_dict[col][1]
                    rfr = RandomForestRegressor(n_estimators=20,n_jobs=-1,max_features=0.6)
                    rfr.fit(frame.loc[index_f_,full_col],frame.loc[index_f_,col])
                    pre = rfr.predict(frame.loc[index_l_,full_col])
                    frame.loc[index_l_,col] = pre
                    
    gc.collect()
    return frame[frame_col]

# 独热编码

In [6]:
def one_hot_str(frame,col,replace=True):
    if replace:
        a_ = frame.pop(col)
    else:
        a_ = frame[col]
    a_.fillna('miss',inplace=True)
    a_ = pd.get_dummies(a_,prefix=a_.name)
    frame = pd.concat([frame,a_],axis=1)
    del a_
    return frame
def one_hot_int(frame,col,number):
    a_ = frame.pop(col)
    a_ = pd.qcut(a_,number)
    col_name_ = [a_.name+'_'+str(i+1) for i in range(number)]
    a_ = pd.get_dummies(a_,)
    a_.columns = col_name_
    frame = pd.concat([frame,a_],axis=1)
    del a_,col_name_
    return frame

# 截取时间节点

In [7]:
def getxyb(date,delta_y = 1,):
    '''得到当前日期的前几年最后一天,默认1年'''
    aaa = date - pd.tseries.offsets.DateOffset(years=1*delta_y)
    return aaa + pd.tseries.offsets.DateOffset(years=1,months=1 - aaa.month, days= - aaa.day)
def getxsb(date,delta_s = 1,):
    '''得到当前日期的前几个季度最后一天,默认1季度'''
    aaa = date - pd.tseries.offsets.DateOffset(months=3*delta_s)
    return aaa + pd.tseries.offsets.DateOffset(months=3 - ((aaa.month - 1) % 3), days=-aaa.day)
def getxhyb(date,delta_hy = 1,):
    '''得到当前日期的前几个半年最后一天,默认1半年'''
    aaa = date - pd.tseries.offsets.DateOffset(months=6*delta_hy)
    return aaa + pd.tseries.offsets.DateOffset(months=6 - ((aaa.month - 1) % 6), days=-aaa.day)

# 原始财务数据读取+分布图

In [8]:
start_year = 2013
end_year = 2018
data_a = DataFrame()
for i in range(start_year,end_year+1):
    if i % 3 ==0:
        print('is concating {} {}/{}'.format(i,i-start_year+1,end_year+1-start_year))
    try:
        path_a = path+f'y/{i}y.xlsx'
        data_a_ = pd.read_excel(path_a)[:-2]
        data_a_.drop(['是否经过审计','审计意见']+[i for i in data_a_.columns if i.find('E')!=-1],axis=1,inplace=True)
        data_a = pd.concat([data_a,data_a_])
    except:
        print(f'no {i}')
        pass
del data_a_
gc.collect()
print('finish concat data_y')

print(np.array(list(data_a.isnull().sum(0))))
data_a.dropna(thresh=data_a.shape[1]-6,inplace=True)
data_a['n_loss'] = data_a.isnull().sum(1)

is concating 2013 1/6
is concating 2016 4/6
finish concat data_y
[   0    0  178 2569  232  420 3327 3327  810 1083  980  397  735  578
  933 1634 3343  937 2266 3128 4122  850 3333 3334 3186 2575 8015]


In [9]:
# for k in range(2):
#     col_ = np.random.choice(data_a.columns[2:-1],len(data_a.columns[2:-1]),replace=False,)
#     for j in col_:
#         data_a = drop_out(data_a,j,model='gauss',alpha=3)

In [10]:
# for i in range(data_a.shape[1]-3):
#     ax = plt.subplot(5,5,i+1)
#     ax.scatter(range(data_a.shape[0]),data_a.iloc[:,i+2].sort_values(),s=3)
#     plt.title(data_a.columns[i+2])
# plt.suptitle('散点趋势图')

# 公司信息读取

In [12]:
indestry = pd.read_excel(path+'comp_feature/产业类发债企业行业分类0910.xlsx',sheet_name='城投类企业')

all_com = indestry[['名称','最新评级', '企业性质', '是否上市','一级分类', '二级分类']]

In [13]:
list_company_property = ['民营企业','地方国有企业','中央国有企业']
all_com['企业性质'] = all_com['企业性质'].map(lambda x:x if x in list_company_property else '其他')

# 违约信息读取+筛选第一次

In [14]:
list_re_col = ['发生日期','名称',]#'发行时主体评级'

re_of_de = pd.read_excel(path+'report of defaulted.xlsx',)[:-2]
re_of_de = re_of_de[list_re_col]
gc.collect()
print('违约记录数:',re_of_de['名称'].shape[0])
re_of_de = re_of_de.groupby(['名称',],as_index=False).apply(lambda x:x.sort_values(['发生日期']).iloc[0])
print('违约公司数:',re_of_de['名称'].shape[0])
all_com['whetherin'] = all_com['名称'].isin(re_of_de['名称'])*1  ##  总表中的违约公司
print('总表里的违约公司数:',sum(all_com['whetherin']))
re_of_de['whetherin'] = re_of_de['名称'].isin(all_com['名称'])*1  ##   在总表中有记录的违约公司
print('缺失数:',re_of_de['名称'].shape[0]-sum(all_com['whetherin']))

loss_re_name = re_of_de.loc[re_of_de['whetherin']==0,'名称']
print('缺失的公司名如下:\n---------------------------\n',pd.unique(loss_re_name),re_of_de.shape)

re_of_de = re_of_de.loc[re_of_de['whetherin']==1,:]

re_of_de = re_of_de.iloc[:,:-1]
all_com = all_com.iloc[:,:-1]

违约记录数: 181
违约公司数: 74
总表里的违约公司数: 0
缺失数: 74
缺失的公司名如下:
---------------------------
 ['上海云峰(集团)有限公司' '上海华信国际集团有限公司' '上海市建设机电安装有限公司' '东兴金满堂商贸有限公司'
 '东北特殊钢集团有限责任公司' '东飞马佐里纺机有限公司' '中伟(徐州)新型材料科技有限公司' '中国中钢股份有限公司'
 '中国城市建设控股集团有限公司' '中国第二重型机械集团有限公司' '中安科股份有限公司' '中恒通(福建)机械制造有限公司'
 '中成新星油田工程技术服务股份有限公司' '中煤集团山西华昱能源有限公司' '中科云网科技集团股份有限公司' '中融双创(北京)科技集团有限公司'
 '丹东港集团有限公司' '五洋建设集团股份有限公司' '亚邦投资控股集团有限公司' '亿利资源集团有限公司' '亿阳集团股份有限公司'
 '保定天威英利新能源有限公司' '保定天威集团有限公司' '信阳市弘昌管道燃气工程有限责任公司' '内蒙古博源控股集团有限公司'
 '内蒙古奈伦农业科技股份有限公司' '内蒙古奈伦集团股份有限公司' '凯迪生态环境科技股份有限公司' '华珠(泉州)鞋业有限公司'
 '华盛江泉集团有限公司' '协鑫集成科技股份有限公司' '南京雨润食品有限公司' '吉林粮食集团收储经销有限公司' '四川圣达集团有限公司'
 '四川省煤炭产业集团有限责任公司' '大连机床集团有限责任公司' '天津市天联滨海复合材料有限公司' '天津市泰亨气体有限公司'
 '安徽蓝博旺机械集团' '宿迁市致富皮业有限公司' '富贵鸟股份有限公司' '山东山水水泥集团有限公司' '山东滨州新天阳化工有限责任公司'
 '山东玖龙海洋产业股份有限公司' '山东迪浩耐磨管道股份有限公司' '广州华工百川科技有限公司' '广西有色金属集团有限公司'
 '惠州侨兴电信工业有限公司' '惠州侨兴电讯工业有限公司' '春和集团有限公司' '武汉国裕物流产业集团有限公司' '江苏中联物流股份有限公司'
 '江苏保千里视像科技集团股份有限公司' '江苏大宏纺织集团股份有限公司' '河北省物流产业集团有限公司' '河南佳源乳业股份有限公司'
 '波鸿集团有限公司' '浙江平湖华龙实业股份有限公司' '淄博宏达矿

# 时光机器

In [14]:
re_of_de['发生日期'] = re_of_de['发生日期'].map(lambda x:getxyb(x))

# 处理财报数据,如果要画图就在这里改

# 切割数据

In [15]:
# data_a = data_a.loc[data_a['名称'].isin(all_com['名称']),:]

# re_of_de = re_of_de.merge(data_a,on=['名称',],)  #留下违约公司所有财报数据re_of_de
# re_of_de = re_of_de.loc[re_of_de['发生日期'] >= re_of_de['报告期'],:]
# re_of_de.insert(0,'target',(re_of_de['发生日期'].dt.year - re_of_de['报告期'].dt.year))

# data_a.insert(0,'target',-1)
# fin_col = re_of_de.columns
# data_a = data_a.loc[~data_a['名称'].isin(re_of_de['名称']),:]   #总的数据表中剔除re_of_de

# data_2017 = data_a.loc[data_a['报告期'].dt.year == 2017,:]
# data_a = data_a.loc[data_a['报告期'].dt.year != 2017,:]

# data_a.dropna(inplace=True)  #总表剔除nan

# for k in range(2):
#     col_ = np.random.choice(data_a.columns[3:-1],len(data_a.columns[3:-1]),replace=False,)
#     for j in col_:
#         data_a = drop_out(data_a,j,model='gauss',alpha=3)

# a = pd.concat([re_of_de,data_2017,data_a])[fin_col]

# del re_of_de,data_a
# gc.collect()

# weight_dict = {}
# a.iloc[:,4:-1],weight_dict = diy_ss(a.iloc[:,4:-1],weight_dict,0)

# a = one_hot_str(a,'报告期',replace=False)

# com_flist = ['名称','是否交通','一级分类','企业性质','是否上市']
# a = a.merge(all_com[com_flist],on='名称')

# com_flist_ = com_flist.copy()
# for o in ['名称','一级分类']:
#     com_flist_.remove(o)

# for i in com_flist_:
#     a = one_hot_str(a,i)

# drop_length = 1
# drop_length += len(set(a['报告期']))
# for i in com_flist_:
#     drop_length += len(set(all_com[i]))

# cols_a = list(a)
# cols_a.insert(len(cols_a), cols_a.pop(cols_a.index('一级分类')))
# a = a[cols_a]

# a.iloc[:,4:-1] = fillna_(a.iloc[:,4:-1],a['一级分类'],n_epoch=4)

# def rechange(dataframe,col,weight):
#     if weight[col] == 'log1p':
#         return np.exp(dataframe[col])-1
#     else:
#         return dataframe[col]*weight[col][1]+weight[col][0]

# a = a.iloc[:,:drop_length*-1]
# a.insert(a.shape[1],'净利润/带息债务',rechange(a,'净利润(亿元)',weight_dict)/((rechange(a,'带息债务(亿元)',weight_dict))+0.01))
# a[['净利润/带息债务']],weight_dict = diy_ss(a[['净利润/带息债务']],weight_dict,0)

# a.to_excel(path+'2013-2018_y.xlsx',index=False)
# joblib.dump(weight_dict,path+'weight_dict_y.m')

In [15]:
fin_col = re_of_de.columns

del re_of_de,data_a
gc.collect()

a = pd.read_excel(path+'2013-2018_y.xlsx')
weight_dict = joblib.load(path+'weight_dict_y.m')

# 违约范围划定

In [16]:
a.loc[a['发生日期'].notnull(),'target'] = a.loc[a['发生日期'].notnull(),'target'].map(lambda x: 1 if x <= 0 else 0)

In [17]:
a.drop('发生日期',axis=1,inplace=True)
re_of_de_0 = a.loc[a['target']==0,:]
re_of_de_1 = a.loc[a['target']==1,:]
data_2017 = a.loc[(a['报告期'].dt.year == 2017)&(a['target'] == -1),:]
data_a = a.loc[(a['报告期'].dt.year != 2017)&(a['target'] == -1),:]
data_a['target'] = data_a['target'].map(lambda x:0)
del a
data_a = data_a.loc[data_a['报告期'].between('2015-01-01','2017-12-31')]
gc.collect()

19

# 处理公司非财报数据

In [18]:
all_com = all_com[['名称','是否交通','企业性质','是否上市',]]    #'一级分类','二级分类'
all_com = one_hot_str(all_com,'企业性质')
all_com = one_hot_str(all_com,'是否上市')
# all_com = one_hot_str(all_com,'所属省市')
# all_com = one_hot_int(all_com,'注册资本(万元)',number=3)

re_of_de_0 = re_of_de_0.merge(all_com,on='名称',)
re_of_de_1 = re_of_de_1.merge(all_com,on='名称',)
data_2017 = data_2017.merge(all_com,on='名称',)
data_a = data_a.merge(all_com,on='名称')
del all_com

# 看看corr

In [21]:
a = [('净资产回报率(%)', 0.2831613389507573),
 ('净利润(亿元)', 0.19612521767114766),
 ('流动比率', 0.10006272874771727),
 ('净资产(亿元)', 0.07643978218479412),
 ('主营业务利润(亿元)', 0.06485263851265907),
 ('货币资金/短期债务', 0.04648683392338189),
 ('总资产报酬率(%)', 0.03358045966497278),
 ('筹资活动现金流(亿元)', 0.0299973498895215),
 ('投资活动现金流(亿元)', 0.02486080556388141),
 ('总资产(亿元)', 0.02458365655281366),
 ('短期债务/总债务', 0.023363955568147503),
 ('经营活动现金流(亿元)', 0.01991941294740471),
 ('速动比率', 0.017362472591467188),
 ('货币资产(亿元)', 0.013813465657292835),
 ('带息债务(亿元)', 0.012756002097363587),
 ('主营业务收入(亿元)', 0.00853891335415696),
 ('净债务(亿元)', 0.008103904697081556),
 ('主营业务利润率(%)', 0.005525472849658592),
 ('资产负债率', 0.0040236587366119),
 ('存货周转率', 0.002683313003464182),
 ('总债务(亿元)', 0.002371458968943401),]
pd.concat([re_of_de,data_2018,data_a])[[i[0] for i in a]].corr().to_excel(path+'corr.xlsx',)

In [13]:
# list_choich = ['名称', '报告期','获息倍数','货币资金/短期债务','净利润亿元','筹资活动现金流亿元','主营业务利润率%',
#                 '主营业务收入增长率%','投资活动现金流亿元','净资产回报率%','总资产报酬率%','流动比率']


In [14]:
# for i in [re_of_de,data_2018]:
#     i.fillna(0,inplace=True)

# 把标签特征移入data_object

In [19]:
# object_list = ['名称', '报告期','一级分类','二级分类']
object_list = ['名称', '报告期',]
data_a_object = pd.DataFrame();data_2017_object = pd.DataFrame();re_object_0 = pd.DataFrame();re_object_1 = pd.DataFrame()

for i in object_list:
    data_a_object = pd.concat([data_a_object,data_a.pop(i)],axis=1)
for i in object_list:
    data_2017_object = pd.concat([data_2017_object,data_2017.pop(i)],axis=1)
for i in object_list:
    re_object_0 = pd.concat([re_object_0,re_of_de_0.pop(i)],axis=1)
for i in object_list:
    re_object_1 = pd.concat([re_object_1,re_of_de_1.pop(i)],axis=1)

re_target_0 = re_of_de_0.pop('target')

re_target_0 = re_target_0.map(lambda x : 0)
re_target_1 = re_of_de_1.pop('target')
data_a_target = data_a.pop('target')
data_2017_target = data_2017.pop('target')

# 数据集 -> x1,x2,y1,y2

In [20]:
def get_hxy(n_splits,length_0=0.6,length_a=0.022,test_size=0.3,simple=False,random_state=None):
    '''横向数据'''
    global list_
    list_ = re_of_de_1.columns
    index_a = np.random.permutation(len(data_a_target))[:int(len(data_a_target)*length_a)]
    x = pd.concat([re_of_de_1,data_a.iloc[index_a,:]],); y = np.r_[re_target_1,[0]*(len(index_a))]
    x.index = range(len(x))
    ss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
#     for train_index, test_index in ss.split(x, y):
#         x1, x2 = x.iloc[train_index,:], x.iloc[test_index,:]
#         y1, y2 = y[train_index], y[test_index]
#     x1,x2,y1,y2 = train_test_split(pd.concat([re_of_de_1,data_a.iloc[index_a,:]],),
#                                    np.r_[re_target_1,[0]*(len(index_a))],test_size=test_size,random_state=random_state)
    if simple:
        pass
    return ss, x, y

def get_zxy(length_0=0.6,test_size=0.3,simple=False,random_state=None):
    '''纵向数据'''
    global x1,x2,y1,y2,list_
    list_ = re_of_de_1.columns
    index_0 = np.random.permutation(len(re_target_0))[:int(len(re_target_0)*length_0)]
    x1,x2,y1,y2 = train_test_split(pd.concat([re_of_de_1,re_of_de_0.iloc[index_0,:],],),
                                   np.r_[re_target_1,[0]*(len(index_0))],test_size=test_size,random_state=random_state)
    if simple:
        list_ = ['获息倍数' , '货币资金/短期债务' , '净利润(亿元)' , '筹资活动现金流(亿元)' ,
                 '主营业务利润率(%)' , '主营业务收入增长率(%)' , '投资活动现金流(亿元)' ,
                 '净资产回报率(%)' , '总资产报酬率(%)' , '流动比率' , '净利润/带息债务']
        x1 = x1[list_]
        x2 = x2[list_]

# hpsklearn

In [21]:
from functools import reduce

In [22]:
from hyperopt import tpe,fmin,hp
from hpsklearn import random_forest,HyperoptEstimator

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


## random_forset

In [68]:
#rf
def model_score(args):
    n_estimators ,max_depth ,min_samples_split ,max_features = args
    model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                   class_weight='balanced',n_jobs=-1,
                                   min_samples_split=min_samples_split,max_features=max_features,)
    model.fit(x1,y1)
    pre = model.predict(x2)
    recall_ = metrics.recall_score(pre,y2)
    accu_ = metrics.accuracy_score(pre,y2)
    auc_ = metrics.roc_auc_score(pre,y2)
    score_ = -(recall_*0.6+auc_*0.2+accu_*0.2)
    return score_

space_dict = {
              'n_estimators':[i for i in range(30,90,5)],
              'max_depth':[2,3,4,],
              'min_samples_split':[i for i in range(2,11,2)],
              'max_features':[i/100 for i in range(50,71,5)],
              }
space = [hp.choice(i,space_dict[i]) for i in space_dict.keys()]

def diy_hyporept(epoch, method, simple, index_number = None, n_spilts = 10):
    result_list = []
    if method == 'z':
        for i in range(epoch):
            get_zxy(simple=simple, random_state=i*3)
            index = fmin(model_score,space,tpe.suggest,
                        reduce(lambda x,y:x*y, [len(i) for i in space_dict.values()]) if index_number == None else index_number)

            result_dict = {}
            for i in index.keys():
                result_dict[i] = space_dict[i][index[i]]
            rfc = RandomForestClassifier(n_estimators=result_dict['n_estimators'],
                                     max_depth=result_dict['max_depth'],
                                     min_samples_split=result_dict['min_samples_split'],
                                     max_features=result_dict['max_features'],
                                     class_weight='balanced',n_jobs=-1,)
            global x1,x2,y1,y2
            rfc.fit(x1,y1)
            result_list.append((rfc,result_dict))
    
    elif method == 'h':
        for i in range(int(np.sqrt(epoch))+1):
            ss, x, y = get_hxy(n_splits=int(np.sqrt(epoch)),)
            for train_index, test_index in ss.split(x, y):
                x1, x2 = x.iloc[train_index,:], x.iloc[test_index,:]
                y1, y2 = y[train_index], y[test_index]
                index = fmin(model_score,space,tpe.suggest,
                        reduce(lambda x,y:x*y, [len(i) for i in space_dict.values()]) if index_number == None else index_number)

                result_dict = {}
                for i in index.keys():
                    result_dict[i] = space_dict[i][index[i]]
                rfc = RandomForestClassifier(n_estimators=result_dict['n_estimators'],
                                         max_depth=result_dict['max_depth'],
                                         min_samples_split=result_dict['min_samples_split'],
                                         max_features=result_dict['max_features'],
                                         class_weight='balanced',n_jobs=-1,)
                rfc.fit(x1, y1)
                result_list.append((rfc,result_dict))
    return result_list

In [71]:
result_list = diy_hyporept(300,'h',True,30)

In [ ]:
#lr
def model_score(args):
    penalty, tol, C = args
    model = LogisticRegression(penalty=penalty,tol=tol,C=C,n_jobs=-1,)
    model.fit(x1,y1)
    pre = model.predict(x2)
    recall_ = metrics.recall_score(pre,y2)
    accu_ = metrics.accuracy_score(pre,y2)
    auc_ = metrics.roc_auc_score(pre,y2)
    score_ = -(recall_*0.25+auc_*0.5+accu_*0.25)
    return score_

space_dict = {
              'penalty':['l1', 'l2'],
              'tol':[0.0001*i for i in [0.1,0.2,0.5,1,2,5,10]],
              'C':[i/10 for i in range(1,100,9)],
              }
space = [hp.choice(i,space_dict[i]) for i in space_dict.keys()]

result_list = []
for epoch in range(epoch):
    get_zxy(simple=True)
    print(f'epoch:{i}, x1:{x1.values[0][0]}')
#     index = fmin(model_score,space,tpe.suggest,30)
    index = fmin(model_score,space,tpe.suggest,reduce(lambda x,y:x*y, [len(i) for i in space_dict.values()]))

    result_dict = {}
    for i in index.keys():
        result_dict[i] = space_dict[i][index[i]]
    lr = LogisticRegression(penalty=result_dict['penalty'],
                             tol=result_dict['tol'],
                             C=result_dict['C'],n_jobs=-1,)
    result_list.append((lr,result_dict))

In [72]:
for step,(i,_) in enumerate(result_list):
# metrics.recall_score(y2,rfc.predict(x2))
    print(0.6*metrics.recall_score(y2,i.predict(x2))+\
          0.2*metrics.roc_auc_score(y2,i.predict(x2))+\
          0.2*metrics.accuracy_score(y2,i.predict(x2)))

0.9093028322440088
0.8960130718954249
0.9093028322440088
0.9468409586056645
0.9093028322440088
0.9734204793028323
0.9093028322440088
0.8318954248366013
0.882723311546841
0.9734204793028323
0.8960130718954249
0.9734204793028323
0.9734204793028323
0.9093028322440088
0.8960130718954249
0.8053159041394335
0.8960130718954249
0.9734204793028323
0.9093028322440088
0.8451851851851853
0.8186056644880174
0.9093028322440088
0.8186056644880174
0.9093028322440088
0.9093028322440088
0.8186056644880174
0.9734204793028323
0.986710239651416
0.9225925925925926
0.8960130718954249
0.9734204793028323
0.9225925925925926


KeyboardInterrupt: 

In [30]:
data_a[list_]

,获息倍数,货币资金/短期债务,净利润(亿元),筹资活动现金流(亿元),主营业务利润率(%),主营业务收入增长率(%),投资活动现金流(亿元),净资产回报率(%),总资产报酬率(%),流动比率,净利润/带息债务
0,-0.042642,0.301501,-0.125115,-0.409274,-0.031412,-0.055219,0.061723,-0.052926,-0.671301,0.645951,-0.029924
1,-0.040961,0.197871,-0.110328,-0.150873,-0.017713,-0.043730,0.141273,-0.135119,-0.618464,0.615402,-0.029246
2,-0.029702,0.259468,-0.071732,-0.051090,0.008620,0.012826,0.065669,0.195567,0.986173,0.528214,-0.024853
3,-0.033278,0.113832,-0.074549,0.055570,-0.000094,-0.017352,0.003983,0.134983,0.486563,0.312253,-0.025991
4,-0.023184,0.218325,-0.031073,-0.076122,-0.010239,-0.029109,-0.002143,0.056590,-0.323034,1.052045,-0.027793
5,0.002228,0.220345,0.021182,-0.109510,-0.011430,0.046604,0.036853,0.166546,-0.133629,1.038083,-0.026790
6,-0.042895,0.231871,-0.125337,-0.073711,-0.063732,-0.044114,0.098050,-0.116828,-0.909365,1.148513,-0.030266
7,-0.035134,0.459766,-0.077883,0.039450,0.002809,0.069287,0.077994,-0.038461,-0.281960,1.271501,-0.027669
8,-0.037434,0.445464,-0.106896,-0.142564,-0.025359,-0.037200,0.150438,-0.117399,-0.646938,0.970817,-0.027616
9,-0.035507,0.145987,-0.109482,-0.175127,-0.027087,-0.066028,0.129221,-0.137638,-0.922140,1.170933,-0.027914


In [74]:
get_zxy(simple=False)
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    pre = [i[0].predict_proba(eval(j[0])[list_])[:,1] for i in result_list]
    pre = [sum(i)/len(result_list) for i in zip(*pre)]
    pre = [1 if i>=0.5 else 0 for i in pre]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),pre),'\n-----------')

---data_a---
 [[2413  170]
 [   0    0]] 
-----------
---re_of_de_0---
 [[54 47]
 [ 0  0]] 
-----------
---re_of_de_1---
 [[ 0  0]
 [ 3 28]] 
-----------


In [76]:
pre_2017 = [i[0].predict_proba(data_2017[list_])[:,1] for i in result_list]
pre_2017 = [sum(i)/len(result_list) for i in zip(*pre_2017)]
pre_2017_ = [1 if i>=0.5 else 0 for i in pre_2017]

sum(pre_2017_)

672

In [33]:
f = lr.predict(data_2017[list_])

f.sum()

41

In [34]:
result_list

[(LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
            penalty='l2', random_state=None, solver='liblinear', tol=5e-05,
            verbose=0, warm_start=False),
  {'C': 0.1, 'penalty': 'l2', 'tol': 5e-05})]

In [27]:
result_list = joblib.load(r'D:\pycharm_folder\xijing\company\2018-06-07\m_save\2018-06-30_result list.m')

# XGBboost

## 寻找参数

In [144]:
para_test = {'learning_rate': [i/1000 for i in range(1,10,2)],
              'max_depth':[i for i in range(3,8,2)], 
              'min_child_weight':[1,3,5],
              'gamma':[i/10 for i in range(1,50,8)],
              'subsample':[i/100 for i in range(50,80,5)],
              'colsample_bytree':[i/100 for i in range(30,60,5)],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
              'scale_pos_weight':[i for i in range(6,11,2)]}
xgb_ = xgb.XGBClassifier(objective= 'binary:logistic',eval_metric='auc',n_jobs=-1,early_stopping_rounds=30)
flod = KFold(n_splits=4)

grid = GridSearchCV(xgb_,param_grid=para_test,scoring='roc_auc',cv=flod,n_jobs=-1,iid=False, )

In [145]:
feature_importance_mean = pd.Series()
xgb_clone = []
def modelfit(alg,feature_importance_mean,xgb_clone,useTrainCV=True,cv_folds=3, early_stopping_rounds=50, n_epoch=5,):
    get_zxy()
    for i in range(n_epoch):
        if useTrainCV:
            xgb_param = alg.get_xgb_params()
            xgtrain = xgb.DMatrix(x1, label=y1)
            cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                metrics='auc', early_stopping_rounds=early_stopping_rounds,)
            alg.set_params(n_estimators=cvresult.shape[0])
            del xgtrain
            gc.collect()

            if i == 0: #init
                param_test = [{'learning_rate': [0.001,0.004,0.007,0.01]},
                              {'max_depth':[i for i in range(3,8,2)], 'min_child_weight':[1,3,5]},
                              {'gamma':[i/10 for i in range(1,50,2)]},
                              {'subsample':[i/100 for i in range(50,80,3)],'colsample_bytree':[i/100 for i in range(30,60,3)]},
                              {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]},
                              {'scale_pos_weight':[1,2,3,5]}]
                for i in range(len(param_test)):
                    gsearch = GridSearchCV(estimator = alg, param_grid = param_test[i], scoring='roc_auc',iid=False, cv=cv_folds)
                    gsearch.fit(x1,y1)
                    alg = gsearch.best_estimator_

            param_test_little_list = [['learning_rate'],
                                      ['max_depth','min_child_weight'],
                                      ['gamma'],
                                      ['subsample','colsample_bytree'],
                                      ['reg_alpha'],
                                      ['scale_pos_weight']]
            for little in range(len(param_test_little_list)):
                param_test_little_dict = {}
                for j in param_test_little_list[little]:
                    if little == 1:
                        param_test_little_dict[j] = [i for i in range(int(xgb1.get_xgb_params()[j])-1,int(xgb1.get_xgb_params()[j])+2)]
                    elif little == 3:
                        param_test_little_dict[j] = [i/1000. for i in np.linspace(int(1000*round(xgb1.get_xgb_params()[j],2)*0.9),
                                                                      int(1000*round(xgb1.get_xgb_params()[j],2)*1.1),
                                                                      3)]
                    else:
                        param_test_little_dict[j] = [i/1000. for i in np.linspace(int(1000*round(xgb1.get_xgb_params()[j],2)*0.9),
                                                                      int(1000*round(xgb1.get_xgb_params()[j],2)*1.1),
                                                                      5)]
                gsearch = GridSearchCV(estimator = alg, param_grid = param_test_little_dict, scoring='roc_auc',iid=False, cv=cv_folds)
                gsearch.fit(x1,y1)
                alg = gsearch.best_estimator_

    #Fit the algorithm on the data
    alg.fit(x1, y1,eval_metric='auc')

    #Print model report:
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y1, alg.predict_proba(x1)[:,1]))
    print ("AUC Score (Test): %f" % metrics.roc_auc_score(y2, alg.predict_proba(x2)[:,1]))
    print('Recall Score (Test): %f' % metrics.recall_score(y2, alg.predict(x2)))

    feature_importance_mean = feature_importance_mean.add(pd.Series(alg.get_booster().get_fscore()),fill_value=0)
    xgb_clone.append(alg)
    return feature_importance_mean,xgb_clone

xgb1 = XGBClassifier(
 learning_rate =0.001,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=2,
 gamma=0.1,
 subsample=0.8,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
)
feature_importance_mean,xgb_clone = modelfit(xgb1,feature_importance_mean,xgb_clone,useTrainCV=True)

AUC Score (Train): 0.500000
AUC Score (Test): 0.500000
Recall Score (Test): 0.000000


In [169]:
get_zxy()
xgb_clone

[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=0.63, gamma=0.09, learning_rate=0.0,
        max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
        n_estimators=1, n_jobs=1, nthread=None, objective='binary:logistic',
        random_state=0, reg_alpha=0.0, reg_lambda=1, scale_pos_weight=0.9,
        seed=None, silent=True, subsample=0.72)]

## 训练

In [160]:
feature_importance_mean = pd.Series()

xgb_epoch = 3
xgb_list = []

for i in range(xgb_epoch):
    get_zxy()
#     get_zxy(test_size=0.3,simple=True)
    print(x1.iloc[0,0])
    
    num_rounds = 1000 # 迭代次数
    
    params={
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'n_estimators':np.random.choice([i for i in range(30,60)]),
    'gamma':np.random.choice([i/100 for i in range(7,12)]),  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth':np.random.choice([3,4,5],p=[0.35,0.55,0.10]), # 构建树的深度，越大越容易过拟合
    'lambda':np.random.choice([i/10 for i in range(1,10)]),  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample':np.random.choice([i/100 for i in range(70,75)]), # 随机采样训练样本
    'colsample_bytree':np.random.choice([i/100 for i in range(35,45)]), # 生成树时进行的列采样
    'min_child_weight':np.random.choice([i for i in range(2,5)]), 
    'eta': np.random.choice([i/1000 for i in range(1,10)]),
    'eval_metric': 'auc',
    'scale_pos_weight':np.random.choice([3,4,5])
    }
    
    xgtrain = xgb.DMatrix(x1, label=y1)
    xgtest = xgb.DMatrix(x2,label=y2,)

    watchlist = [(xgtrain, 'train'),(xgtest, 'val')]
    model = xgb.train(params,xgtrain,num_rounds,watchlist,early_stopping_rounds=30,verbose_eval=10)
    xgb_list.append(model)
    if i == 0:
        feature_importance_mean = model.get_fscore()
    else:
        for g in model.get_fscore():
            feature_importance_mean[g] = feature_importance_mean.get(g,0) + model.get_fscore()[g]
    print ("AUC Score (Test): %f" % metrics.roc_auc_score(y2, model.predict(xgtest)))
    print('Recall Score (Test): %f' % metrics.recall_score(y2, [1 if i>=0.5 else 0 for i in model.predict(xgtest)]))
    
feature_importance_mean = pd.Series(feature_importance_mean).sort_values()/xgb_epoch
feature_importance_mean.plot(kind='barh', title='Feature Importance Score of Mean({})'.format(xgb_epoch))
plt.ylabel('Feature Importances')

4.939655220762393
[0]	train-auc:0.866848	val-auc:0.459375
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 30 rounds.
[10]	train-auc:0.922283	val-auc:0.521875
[20]	train-auc:0.941304	val-auc:0.5375
[30]	train-auc:0.93913	val-auc:0.53125
[40]	train-auc:0.952174	val-auc:0.54375
[50]	train-auc:0.958696	val-auc:0.56875
[60]	train-auc:0.967391	val-auc:0.58125
[70]	train-auc:0.967391	val-auc:0.5625
[80]	train-auc:0.967391	val-auc:0.58125
Stopping. Best iteration:
[59]	train-auc:0.966304	val-auc:0.5875

AUC Score (Test): 0.575000
Recall Score (Test): 0.625000
2.230562014008577
[0]	train-auc:0.880247	val-auc:0.579487
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 30 rounds.
[10]	train-auc:0.957407	val-auc:0.646154
[20]	train-auc:0.97037	val-auc:0.646154
[30]	train-auc:0.975309	val-auc:0.641026
Stopping. Best iteration:
[3]	train-auc:0.9

Text(-81.1528,0.5,'Feature Importances')

In [20]:
# joblib.dump(xgb_list, "m_save/xgb_z_250_三个月.m")
xgb_list = joblib.load('m_save/xgb_z.m')

## 原数据回测

In [25]:
get_zxy()

In [162]:
xga = xgb.DMatrix(data_a[list_]);xg0 = xgb.DMatrix(re_of_de_0[list_]);xg1 = xgb.DMatrix(re_of_de_1[list_])
for j in zip(['xga','xg0','xg1'],['data_a_target','re_target_0','re_target_1']):
    e = [i.predict(eval(j[0])) for i in xgb_list]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),
                                                   np.where(np.array([sum(i)/len(xgb_list) for i in list(zip(*e))])>=0.5,1,0)),'\n-----------')
del xga,xg0,xg1

---xga---
 [[   0 1814  769]
 [   0    0    0]
 [   0    0    0]] 
-----------
---xg0---
 [[73 28]
 [ 0  0]] 
-----------
---xg1---
 [[ 0  0]
 [ 4 27]] 
-----------


## 预测

In [39]:
data_2018[list_].shape

(2810, 10)

In [24]:
xgb_list[0].feature_names

['获息倍数',
 '货币资金/短期债务',
 '净利润(亿元)',
 '筹资活动现金流(亿元)',
 '主营业务利润率(%)',
 '主营业务收入增长率(%)',
 '投资活动现金流(亿元)',
 '净资产回报率(%)',
 '总资产报酬率(%)',
 '流动比率']

In [51]:
xg_2018 = xgb.DMatrix(data_2017[list_])
e_xgb = [i.predict(xg_2018) for i in xgb_list]
e_xgb = np.array([sum(i)/len(xgb_list) for i in list(zip(*e_xgb))])
f = np.where(e_xgb>=0.5,1,0)
del xg_2018
f.sum()

1054

# 随即森林+栅栏搜索

## 栅栏搜索

In [37]:
rfc = RandomForestClassifier(oob_score=True,class_weight={1:10,0:1})

para_list = {
    'n_estimators':range(30,180,5),
    'max_depth':[2,3,4,],
    'min_samples_split':range(2,11,2),
    'max_features':[i/100 for i in range(50,71,5)],
}

flod = KFold(n_splits=4)
grid = GridSearchCV(rfc,param_grid=para_list,scoring='roc_auc',cv=flod,n_jobs=-1)

In [ ]:
grid.fit(x1,y1)

## 保存模型

# joblib.dump(grid, "m_save/违约公司纵向比较全特征_auc.m")

## 读取模型

grid_recall = joblib.load('m_save/违约公司纵向比较全特征_recall.m')
grid_auc = joblib.load('m_save/违约公司纵向比较全特征_auc.m')

## 模型参数

print(grid_auc.best_params_)
print('\n--------\n',grid_auc.best_params_)
grid_auc = grid_auc.best_estimator_
grid_recall = grid_recall.best_estimator_

## 特征重要性

In [262]:
dict_auc = dict(zip(x2.columns,grid_auc.feature_importances_))
dict_recall = dict(zip(x2.columns,grid_recall.feature_importances_))

## 更新训练集、测试集

In [59]:
get_zxy()

## 模型效果在原数据上的混淆矩阵

In [342]:
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    for i in ['grid_auc','grid_recall']:
        print(f'{i}---{j[0]}',metrics.confusion_matrix(eval(j[1]),eval(i).predict(eval(j[0]))),'\n-----------')

grid_auc---data_a [[15558    53]
 [    0     0]] 
-----------
grid_recall---data_a [[15561    50]
 [    0     0]] 
-----------
grid_auc---re_of_de_0 [[437  13]
 [  0   0]] 
-----------
grid_recall---re_of_de_0 [[447   3]
 [  0   0]] 
-----------
grid_auc---re_of_de_1 [[ 0  0]
 [13 13]] 
-----------
grid_recall---re_of_de_1 [[ 0  0]
 [17  9]] 
-----------


## 训练模型 

In [172]:
rfc_list = []
rfc_recall = []
rfc_epoch = 5
for i in range(rfc_epoch):
    get_zxy(test_size=0.25,simple=True)
    rfc = RandomForestClassifier(n_estimators=np.random.choice(range(20,30)),
                                 max_depth=np.random.choice([2,3],p=[0.5,0.5,]),
                                 max_features=np.random.choice([i/100 for i in range(35,55)]),
                                 min_samples_split=np.random.choice([3,4,5],p=[0.15,0.45,0.4]),
                                 n_jobs=-1,class_weight='balanced')
    rfc.fit(x1,y1)
    rfc_recall.append(metrics.recall_score(y2,rfc.predict(x2)))
    rfc_list.append(rfc)
    if i == 0:
        feature_importance = rfc.feature_importances_
    else :
        feature_importance += rfc.feature_importances_
    if i%30 == 0:
        print(i)
feature_importance = [round(i/rfc_epoch,6) for i in feature_importance]
print('avg-recall:',sum(rfc_recall)/rfc_epoch)
pd.Series(dict(zip(x2.columns,feature_importance))).sort_values().plot(kind='barh')

0
avg-recall: 0.3921428571428572


In [168]:
print('\n',np.array(sorted(dict(zip(x2.columns,feature_importance)).items(),key=lambda x:x[1],reverse=True)))


 [['净利润(亿元)' '0.13383']
 ['净利润/带息债务' '0.125542']
 ['货币资金/短期债务' '0.100117']
 ['净资产回报率(%)' '0.097667']
 ['总资产报酬率(%)' '0.094849']
 ['获息倍数' '0.094658']
 ['筹资活动现金流(亿元)' '0.089516']
 ['主营业务利润率(%)' '0.085955']
 ['投资活动现金流(亿元)' '0.060645']
 ['主营业务收入增长率(%)' '0.059243']
 ['流动比率' '0.057979']]


In [ ]:
# joblib.dump(rfc_list, "m_save/rfc_z_250_三个月.m")

In [47]:
rfc_list = joblib.load("m_save/clf_grid12.m")

## 原数据回测

In [44]:
get_hxy()

In [173]:
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    e = [i.predict_proba(eval(j[0])[list_])[:,1] for i in rfc_list]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),
                                                   np.where(np.array([sum(i)/rfc_epoch for i in list(zip(*e))])>=0.5,1,0)),'\n-----------')
# [ 1 if ((sum(i)/epoch)>=0.5) else 0 for i in list(zip(*e)) ]

---data_a---
 [[   0 2385  198]
 [   0    0    0]
 [   0    0    0]] 
-----------
---re_of_de_0---
 [[93  8]
 [ 0  0]] 
-----------
---re_of_de_1---
 [[ 0  0]
 [10 21]] 
-----------


## 预测

In [171]:
e_rfc = [i.predict_proba(data_2017[list_])[:,1] for i in rfc_list]
e_rfc = np.array([sum(i)/len(rfc_list) for i in list(zip(*e_rfc))])
f = np.where(e_rfc>=0.5,1,0)

f.sum()

164

## LR

In [38]:
para_list = {
    'lr__penalty':['l2','l1'],
    'lr__tol':[i/10000 for i in range(1,101,20)],
    'lr__C':[0.1,0.5,1,5,10],
    'lr__class_weight':[dict([(0,i),(1,1-i)]) for i in np.linspace(0.1,0.5,5)]+['balanced']
}

pp_lr = Pipeline([('pn',PolynomialFeatures()),
                      ('lr',LogisticRegression(fit_intercept=False,))])
nflod = KFold(n_splits=4,shuffle=True)
grid = GridSearchCV(pp_lr,para_list,n_jobs=-1,cv=nflod,scoring='recall')

In [ ]:
get_zxy()
grid.fit(x1,y1)

In [97]:
grid.best_params_

{'lr__C': 1,
 'lr__class_weight': 'balanced',
 'lr__penalty': 'l1',
 'lr__tol': 0.0081}

In [53]:
lr_epoch = 10
lr_list = []
for i in range(lr_epoch):
    get_zxy(length_0=.4,simple=True)
    lr = LogisticRegression(fit_intercept=False,C=np.random.choice([2,4,6],p=[0.3,0.4,0.3]),
                    class_weight=np.random.choice(['balanced',]+[dict([(0,i),(1,1-i)]) for i in np.linspace(0.2,0.4,3)],p=[0.25,0.25,0.25,0.25]),
                    penalty=np.random.choice(['l1','l2'],p=[0.25,0.75]),
                    tol=np.random.choice([i/50 for i in range(74,87,2)],)           )
    lr.fit(x1,y1)
    lr_list.append(lr)
    if i == 0:
        feature_weight = np.array(lr.coef_).reshape(-1)
    else:
        feature_weight += np.array(lr.coef_).reshape(-1)

feature_weight = pd.Series(feature_weight/lr_epoch,index=x2.columns).sort_values()
feature_weight.plot(kind='barh',title=f'lr of epoch({lr_epoch})')

## 回测原数据

In [54]:
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    e = [i.predict_proba(eval(j[0])[list_])[:,1] for i in lr_list]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),
                                                   np.where(np.array([sum(i)/lr_epoch for i in list(zip(*e))])>=0.5,1,0)),'\n-----------')

---data_a---
 [[14195  8879]
 [    0     0]] 
-----------
---re_of_de_0---
 [[187 222]
 [  0   0]] 
-----------
---re_of_de_1---
 [[ 0  0]
 [19 63]] 
-----------


## 预测 

In [55]:
e_lr = [i.predict_proba(data_2018[list_])[:,1] for i in lr_list]
e_lr = np.array([sum(i)/lr_epoch for i in list(zip(*e_lr))])
f = np.where(e_lr>=0.5,1,0)

f.sum()

1037

## 数据提取

In [ ]:
e_lr[data_2018_object['名称'].map(lambda x:x.find('新光')>=0)]

## SVM 

In [27]:
para_list = {'C':[1,5,10],
             'kernel':['linear','poly','rbf'],
             'degree':[2,3]}
svc = SVC(class_weight='balanced',)
nflod = KFold(n_splits=4,shuffle=True)
grid = GridSearchCV(svc,para_list,n_jobs=-1,cv=nflod,scoring='roc_auc')

In [28]:
get_zxy(simple=True)
grid.fit(x1,y1)

GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [1, 5, 10], 'kernel': ['linear', 'poly', 'rbf'], 'degree': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [29]:
grid.best_params_

{'C': 1, 'degree': 2, 'kernel': 'rbf'}

In [50]:
svc_epoch = 10
svc_list = []
for i in range(svc_epoch):
    get_zxy(length_0=.4,simple=True)
    svc = SVC(C=np.random.choice([1,5],p=[0.6,0.4],),class_weight='balanced',
              kernel=str(np.random.choice(['linear','rbf','poly'],p=[0.5,0.25,0.25])),
              degree=np.random.choice([2,3]),probability=True)
    svc.fit(x1,y1,)
    svc_list.append(svc)
#     if i == 0:
#         feature_weight = np.array(lr.coef_).reshape(-1)
#     else:
#         feature_weight += np.array(lr.coef_).reshape(-1)

# feature_weight = pd.Series(feature_weight/lr_epoch,index=x2.columns).sort_values()
# feature_weight.plot(kind='barh',title=f'lr of epoch({lr_epoch})')

## 原数据回测

In [52]:
for j in zip(['data_a','re_of_de_0','re_of_de_1'],['data_a_target','re_target_0','re_target_1']):
    e = [i.predict_proba(eval(j[0])[list_])[:,1] for i in svc_list]
    print(f'---{j[0]}---\n',metrics.confusion_matrix(eval(j[1]),
                                                   np.where(np.array([sum(i)/svc_epoch for i in list(zip(*e))])>=0.5,1,0)),'\n-----------')

---data_a---
 [[22995    79]
 [    0     0]] 
-----------
---re_of_de_0---
 [[409]] 
-----------
---re_of_de_1---
 [[ 0  0]
 [52 30]] 
-----------


## 预测

In [19]:
e_svc = [i.predict_proba(data_2018[list_])[:,1] for i in svc_list]
e_svc = np.array([sum(i)/svc_epoch for i in list(zip(*e_svc))])
f = np.where(e_svc>=0.5,1,0)

f.sum()

NameError: name 'svc_list' is not defined

# -------------------------------------------------

# 提取原数据

In [30]:
index_ = pd.concat([data_a_object,data_2018_object])['名称'].map(lambda x : x.find('新疆生产建设兵团第六师国有资产经营有限责任公司')>=0)

In [31]:
outdata = pd.concat([pd.concat([data_a_object,data_2018_object]).loc[index_,['名称','报告期']],\
                     pd.concat([data_a,data_2018]).loc[index_,:]],axis=1)

In [26]:
# outdata.to_excel(path+'新疆生产建设兵团第六师国有资产经营有限责任公司.xlsx',index=False)

In [32]:
outdata = outdata.iloc[:,:-9]

get_zxy(simple=True)

outdata = outdata[list(outdata.columns[:2])+list_+['经营活动现金流(亿元)']]

outdata.set_index('报告期',inplace=True)

outdata.sort_values('报告期',inplace=True)

In [33]:
outdata

,名称,获息倍数,货币资金/短期债务,净利润(亿元),筹资活动现金流(亿元),主营业务利润率(%),主营业务收入增长率(%),投资活动现金流(亿元),净资产回报率(%),总资产报酬率(%),流动比率,经营活动现金流(亿元)
报告期,,,,,,,,,,,,
2016-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.450960,0.735140,-1.724131,0.842136,-1.322245,6.391666,2.110962,-4.098005,-1.208708,0.962192,-2.916603
2016-06-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.261334,0.745595,-1.140379,-0.911901,-7.320250,-3.102262,2.080562,0.410571,-1.330516,2.003252,-2.543357
2016-12-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-0.833186,-1.630456,-0.712209,-1.891611,-1.396979,-2.214950,1.097175,-1.209295,-0.126586,1.033949,-2.559672
2017-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.221802,-1.735538,-1.250023,0.266761,-1.161610,1.013630,0.907222,-1.587410,-1.781123,0.862620,-2.346397
2017-06-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.053352,-1.820156,-1.170228,-0.783732,-0.991694,1.219125,0.511116,-1.501856,-1.848131,1.292461,-0.702139
2017-09-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-0.653031,-0.939512,-0.974024,1.149206,-0.530799,0.375040,0.438609,-1.412630,-1.804707,2.054206,-2.212696
2018-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.345240,-2.430510,-1.427194,-0.823264,-2.182058,0.752173,0.277939,-1.787180,-2.001986,1.390630,0.313063


In [34]:
a = [('获息倍数', 0.10293085778684695),('货币资金/短期债务', 0.08466747934544402),('净利润(亿元)', 0.08119844386846073),
     ('筹资活动现金流(亿元)', 0.07392047950839306),('主营业务利润率(%)', 0.07373536447871205),('主营业务收入增长率(%)', 0.0681957815907612),
 ('货币资金/总债务', 0.059822816991053754),('投资活动现金流(亿元)', 0.056517651957992715),('净资产回报率(%)', 0.05488686308572023),
 ('总资产报酬率(%)', 0.036039995560997476),('短期债务/总债务', 0.0350832996152875),('EBITDA/营业总收入', 0.034048366402690815),
 ('经营活动现金流(亿元)', 0.0243060183484769),('EBITDA/带息债务', 0.023188518342373905),('流动比率', 0.020910667354810814),
 ('成立日期', 0.0202111066498566),('经营性现金流/EBITDA', 0.019427371835625647),('货币资产(亿元)', 0.018929246296673925),
 ('净债务(亿元)', 0.015673135947545064),('EBITDA(亿元)', 0.013656585077388917),('带息债务/总投入资本', 0.013195889611687595),('主营业务利润(亿元)', 0.012345481772175503),
 ('总资产(亿元)', 0.01071488982344452),('净资产(亿元)', 0.009601482353160072),('总债务(亿元)', 0.009112083719364693),
 ('主营业务收入(亿元)', 0.006851438201812423),('速动比率', 0.005393658415696681),('存货周转率', 0.00521645305170003),
 ('资产负债率', 0.0034618581489861853),('带息债务(亿元)', 0.002858337636445684),('是否上市_否', 0.002687756282129772),
 ('企业性质_民营企业', 0.0012106209382845353),('注册资金低', 0.0),('注册资金中等', 0.0),
 ('注册资金高', 0.0),('企业性质_中外合资企业', 0.0),('企业性质_中央国有企业', 0.0),('企业性质_地方国有企业', 0.0),
 ('企业性质_外商独资企业', 0.0),('企业性质_外资企业', 0.0),('企业性质_集体企业', 0.0),('是否上市_是', 0.0)]
a = dict(a)
a = [a[i] for i in outdata.columns[1:]]
a = [j/sum(a) for j in a]
outdata['加权平均'] = outdata.iloc[:,1:].values.dot(np.array(a).reshape(-1,1))

In [53]:
z = pd.read_excel(path+'2017a.xlsx')

In [54]:
z = z[list(z.columns[:2])+list_+['经营活动现金流(亿元)']]

In [55]:
z = z.loc[z['名称']=='新疆生产建设兵团第六师国有资产经营有限责任公司',:]

In [56]:
z = z.iloc[0,:]

In [57]:
z.fillna(1.3,inplace=True)

In [44]:
z

名称              新疆生产建设兵团第六师国有资产经营有限责任公司
报告期                 2017-12-31 00:00:00
获息倍数                                1.3
货币资金/短期债务                     0.0820865
净利润(亿元)                        -3.50252
筹资活动现金流(亿元)                     11.2072
主营业务利润率(%)                      -8.3078
主营业务收入增长率(%)                    14.6426
投资活动现金流(亿元)                  -0.0295982
净资产回报率(%)                        0.2535
总资产报酬率(%)                       -1.1393
流动比率                             2.1238
经营活动现金流(亿元)                    -15.1898
Name: 3937, dtype: object

In [58]:
z = pd.DataFrame(z).stack().unstack(0).set_index('报告期',)

In [59]:
weight = joblib.load(path+'weight_dict_a.m')

In [60]:
for i in range(z.shape[1]-1):
    z.iloc[:,i+1] =(z.iloc[:,i+1]-weight[z.columns[i+1]][0])/weight[z.columns[i+1]][1]

In [61]:
z

,名称,获息倍数,货币资金/短期债务,净利润(亿元),筹资活动现金流(亿元),主营业务利润率(%),主营业务收入增长率(%),投资活动现金流(亿元),净资产回报率(%),总资产报酬率(%),流动比率,经营活动现金流(亿元)
报告期,,,,,,,,,,,,
2017-12-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.1993,-2.42139,-22.5031,1.45793,-3.34059,0.493806,1.10189,-1.43402,-3.18212,1.50703,-6.27212


In [52]:
outdata

,名称,获息倍数,货币资金/短期债务,净利润(亿元),筹资活动现金流(亿元),主营业务利润率(%),主营业务收入增长率(%),投资活动现金流(亿元),净资产回报率(%),总资产报酬率(%),流动比率,经营活动现金流(亿元),加权平均
报告期,,,,,,,,,,,,,
2016-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.450960,0.735140,-1.724131,0.842136,-1.322245,6.391666,2.110962,-4.098005,-1.208708,0.962192,-2.916603,-0.039001
2016-06-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.261334,0.745595,-1.140379,-0.911901,-7.320250,-3.102262,2.080562,0.410571,-1.330516,2.003252,-2.543357,-1.337332
2016-12-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-0.833186,-1.630456,-0.712209,-1.891611,-1.396979,-2.214950,1.097175,-1.209295,-0.126586,1.033949,-2.559672,-1.070478
2017-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.221802,-1.735538,-1.250023,0.266761,-1.161610,1.013630,0.907222,-1.587410,-1.781123,0.862620,-2.346397,-0.753055
2017-06-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.053352,-1.820156,-1.170228,-0.783732,-0.991694,1.219125,0.511116,-1.501856,-1.848131,1.292461,-0.702139,-0.761336
2017-09-30,新疆生产建设兵团第六师国有资产经营有限责任公司,-0.653031,-0.939512,-0.974024,1.149206,-0.530799,0.375040,0.438609,-1.412630,-1.804707,2.054206,-2.212696,-0.417947
2018-03-31,新疆生产建设兵团第六师国有资产经营有限责任公司,-1.345240,-2.430510,-1.427194,-0.823264,-2.182058,0.752173,0.277939,-1.787180,-2.001986,1.390630,0.313063,-1.105021


In [62]:
z['加权平均'] = z.iloc[:,1:].values.dot(np.array(a).reshape(-1,1))

In [63]:
z = pd.concat([outdata,z])[outdata.columns]

In [64]:
outdata = z.sort_index()

In [65]:
for i in range(outdata.shape[1]-1):
    ax = plt.subplot(4,3,i+1)
    outdata.iloc[:,i+1].plot(label=outdata.columns[i+1],)
    ax.legend(loc='best',prop={'size':11})
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.set_xlabel('')
plt.suptitle('新疆生产建设兵团第六师国有资产经营有限责任公司')
plt.show()

# 画图 

## -------------

In [ ]:
data_object['result'] = a.predict(data_2018)

In [ ]:
data_object[data_object['result']==1].shape

In [ ]:
a = dict(a)
a = [j for j in a.values()][:10]+[a['经营活动现金流(亿元)']]
a = [j/sum(a) for j in a]

In [ ]:
def pct_ch(dataframe):
    for i in range(dataframe.shape[1]):
        name = dataframe.columns[i]+'_rate'
        dataframe[name] = pd.Series()
        for j in range(dataframe.shape[0]-1):
            dataframe.iloc[j+1,-1] = (dataframe.iloc[j+1,i]-dataframe.iloc[j,i])/abs(dataframe.iloc[j,i])
    return dataframe.iloc[1:,:]

b_median = b_.iloc[:,1:].groupby('报告期').median().iloc[::-1,:]
b_median['加权平均'] = b_median.values.dot(np.array(a).reshape(-1,1))
b_median = pct_ch(b_median)
b_median.index = [i for i in range(-b_median.shape[0]+1,0+1)]

b_mean = b_.iloc[:,1:].groupby('报告期').mean().iloc[::-1,:]
b_mean['加权平均'] = b_mean.values.dot(np.array(a).reshape(-1,1))
b_mean = pct_ch(b_mean)
b_mean.index = [i for i in range(-b_mean.shape[0]+1,0+1)]

lenght = int(prepared_frame.shape[1]/2)

prepared_frame = b_median
plt.figure()
for i in range(lenght):
    ax = plt.subplot(4,3,i+1)
    prepared_frame.iloc[:,i].plot(label=prepared_frame.columns[i+lenght],)
    ax.legend(loc='best',prop={'size':11})
    for X,Y,Z in zip(prepared_frame.index,prepared_frame.iloc[:,i],prepared_frame.iloc[:,i+lenght]):
        plt.text(X,Y,'%.2f'%Z,ha='center',va='bottom')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    if i == 1:
        plt.title('趋势变化图(中位数)',fontdict={'size':20})
plt.show()

prepared_frame = b_mean
plt.figure()
for i in range(lenght):
    ax = plt.subplot(4,3,i+1)
    prepared_frame.iloc[:,i].plot(label=prepared_frame.columns[i],)
    ax.legend(loc='best',prop={'size':11})
    for X,Y,Z in zip(prepared_frame.index,prepared_frame.iloc[:,i],prepared_frame.iloc[:,i+lenght]):
        plt.text(X,Y,'%.2f'%Z,ha='center',va='bottom')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    if i == 1:
        plt.title('趋势变化图(平均数)',fontdict={'size':20})
plt.show()

In [ ]:
b_median.to_excel(path+'median_.xlsx',index=False)
b_mean.to_excel(path+'mean_.xlsx',index=False)

In [ ]:
re_of_de = pd.read_excel(path+'report of defaulted.xlsx',)[:-2]

In [ ]:
pd.DataFrame(re_of_de['发生日期'].groupby(re_of_de['名称'],).apply(lambda x:x.sort_values('发生日期').iloc[0]))

In [68]:
pd.concat([pd.concat([re_object_0,re_of_de_0],axis=1),pd.concat([re_object_1,re_of_de_1],axis=1)]).drop_duplicates(['名称'])['企业性质_中央国有企业'].sum()

4

In [1]:
a = [[1,0,0],[1,0,1],[0,1,0]]

In [5]:
from functools import reduce

In [11]:
reduce(lambda x,y:np.array(x)|np.array(y),a)

array([1, 1, 1], dtype=int32)

In [107]:
c = pd.DataFrame(np.arange(9).reshape(3,3),columns=['a','b','c'])

In [115]:
c.iloc[0:2,1] = np.nan

In [118]:
c.iloc[0:2,1][0] = 1

In [164]:
c.loc[aa,'b'] = 3

In [165]:
c

,a,b,c
0,0,1.0,2
1,3,3.0,5
2,6,7.0,8


In [161]:
def return_index(aa,bb):
    j=0
    for i in range(len(aa)):
        if aa[i] == False :
            pass
        else:
            aa[i] = bb[j]
            j += 1
    return aa

In [171]:
aa = np.random.choice([True,False],50)
bb = np.random.choice([True,False],sum(aa==True))

In [172]:
aa

array([False,  True,  True,  True, False, False,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
        True, False,  True,  True,  True,  True, False,  True, False,
       False, False,  True, False, False, False,  True,  True, False,
        True,  True,  True, False, False])

In [173]:
bb

array([False, False,  True, False, False,  True,  True,  True, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,  True, False,  True,  True,  True, False,  True,
        True,  True])

In [175]:
return_index(aa,~bb)

array([False, False, False,  True, False, False, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True,  True,  True, False, False])